In [1]:
import pandas as pd
import re

## Wrangling for the first appearance

In [2]:
data_df = pd.read_csv("../data/2022_sd_genomic_metadata.csv",header=0)
data_df["zipcode"] = data_df["zipcode"].astype("str")
data_df = data_df[data_df['nextclade_pango'].isin(['BA.2','BA.2.12.1','BA.4','BA.5'])]
data_df['covv_collection_date'] = pd.to_datetime(data_df['covv_collection_date'])
data_df['week'] = data_df['covv_collection_date'].dt.isocalendar().week
data_df

,covv_virus_name,covv_accession_id,covv_collection_date,nextclade_pango,clade,zipcode,week
0,hCoV-19/USA/CA-SEARCH-123653/2022,EPI_ISL_14001552,2022-06-21,BA.2.12.1,22C (Omicron),92129,25
1,hCoV-19/USA/CA-SEARCH-123678/2022,EPI_ISL_14001569,2022-06-21,BA.5,22B (Omicron),92126,25
2,hCoV-19/USA/CA-SEARCH-123666/2022,EPI_ISL_14001563,2022-06-21,BA.2.12.1,22C (Omicron),92119,25
3,hCoV-19/USA/CA-SEARCH-123649/2022,EPI_ISL_14001549,2022-06-21,BA.5,22B (Omicron),92117,25
4,hCoV-19/USA/CA-SEARCH-123641/2022,EPI_ISL_14001545,2022-06-21,BA.4,22A (Omicron),92104,25
...,...,...,...,...,...,...,...
4978,hCoV-19/USA/CA-SEARCH-75725/2022,EPI_ISL_11103260,2022-01-18,BA.2,21L (Omicron),92109,3
9790,hCoV-19/USA/CA-SEARCH-80026/2022,EPI_ISL_11534696,2022-01-05,BA.2,21L (Omicron),92122,1
10128,hCoV-19/USA/CA-SEARCH-76579/2022,EPI_ISL_11146567,2022-01-05,BA.2,21L (Omicron),92071,1
10645,hCoV-19/USA/CA-SEARCH-114975/2022,EPI_ISL_9196061,2022-01-04,BA.2,21L (Omicron),92127,1


In [3]:
# merge the data_df with community data
community_df = pd.read_csv('/data/bp-codeathon-team-2/zipcode_community.csv', index_col = False, header=0)
community_df['zipcode'] = community_df['zipcode'].astype('str')
community_df

,zipcode,community
0,91901,Alpine
1,91902,Bonita
2,91905,Boulevard
3,91906,Campo
4,91910,Chula Vista
...,...,...
95,92129,San Diego
96,92130,San Diego
97,92131,San Diego
98,92173,San Ysidro


In [4]:
zip_community_merge = data_df.merge(community_df, how='left', on = 'zipcode')
zip_community_merge

,covv_virus_name,covv_accession_id,covv_collection_date,nextclade_pango,clade,zipcode,week,community
0,hCoV-19/USA/CA-SEARCH-123653/2022,EPI_ISL_14001552,2022-06-21,BA.2.12.1,22C (Omicron),92129,25,San Diego
1,hCoV-19/USA/CA-SEARCH-123678/2022,EPI_ISL_14001569,2022-06-21,BA.5,22B (Omicron),92126,25,San Diego
2,hCoV-19/USA/CA-SEARCH-123666/2022,EPI_ISL_14001563,2022-06-21,BA.2.12.1,22C (Omicron),92119,25,San Diego
3,hCoV-19/USA/CA-SEARCH-123649/2022,EPI_ISL_14001549,2022-06-21,BA.5,22B (Omicron),92117,25,San Diego
4,hCoV-19/USA/CA-SEARCH-123641/2022,EPI_ISL_14001545,2022-06-21,BA.4,22A (Omicron),92104,25,San Diego
...,...,...,...,...,...,...,...,...
1028,hCoV-19/USA/CA-SEARCH-75725/2022,EPI_ISL_11103260,2022-01-18,BA.2,21L (Omicron),92109,3,San Diego
1029,hCoV-19/USA/CA-SEARCH-80026/2022,EPI_ISL_11534696,2022-01-05,BA.2,21L (Omicron),92122,1,San Diego
1030,hCoV-19/USA/CA-SEARCH-76579/2022,EPI_ISL_11146567,2022-01-05,BA.2,21L (Omicron),92071,1,Santee
1031,hCoV-19/USA/CA-SEARCH-114975/2022,EPI_ISL_9196061,2022-01-04,BA.2,21L (Omicron),92127,1,San Diego


In [13]:
grouped_data = zip_community_merge.groupby(['community', 'nextclade_pango'])['week'].min().reset_index().rename(columns={"week":"week_appeared"})
grouped_data

,community,nextclade_pango,week_appeared
0,Alpine,BA.2.12.1,19
1,Alpine,BA.4,24
2,Alpine,BA.5,23
3,Bonita,BA.2.12.1,19
4,Bonita,BA.5,23
...,...,...,...
85,Valley Center,BA.2.12.1,20
86,Valley Center,BA.5,21
87,Vista,BA.2,9
88,Vista,BA.2.12.1,19


In [14]:
grouped_data.community.value_counts()

Lakeside              4
La Mesa               4
Spring Valley         4
Santee                4
San Ysidro            4
Chula Vista           4
San Diego             4
El Cajon              4
Escondido             4
National City         4
Imperial Beach        4
San Marcos            3
Ramona                3
Poway                 3
Lemon Grove           3
Alpine                3
La Jolla              2
Bonita                2
Vista                 2
Valley Center         2
Oceanside             2
Campo                 2
Encinitas             2
Jamul                 2
Rancho Santa Fe       2
Coronado              2
Cardiff by the Sea    1
Boulevard             1
Santa Ysabel          1
Dulzura               1
Carlsbad              1
Pauma Valley          1
Palomar Mountain      1
Pala                  1
Fallbrook             1
Jacumba               1
Warner Springs        1
Name: community, dtype: int64

In [15]:
grouped_data.to_csv('../data/first_appearance.csv')

## Wrangling for the counts per week by zipcode

In [16]:
zip_community_merge

,covv_virus_name,covv_accession_id,covv_collection_date,nextclade_pango,clade,zipcode,week,community
0,hCoV-19/USA/CA-SEARCH-123653/2022,EPI_ISL_14001552,2022-06-21,BA.2.12.1,22C (Omicron),92129,25,San Diego
1,hCoV-19/USA/CA-SEARCH-123678/2022,EPI_ISL_14001569,2022-06-21,BA.5,22B (Omicron),92126,25,San Diego
2,hCoV-19/USA/CA-SEARCH-123666/2022,EPI_ISL_14001563,2022-06-21,BA.2.12.1,22C (Omicron),92119,25,San Diego
3,hCoV-19/USA/CA-SEARCH-123649/2022,EPI_ISL_14001549,2022-06-21,BA.5,22B (Omicron),92117,25,San Diego
4,hCoV-19/USA/CA-SEARCH-123641/2022,EPI_ISL_14001545,2022-06-21,BA.4,22A (Omicron),92104,25,San Diego
...,...,...,...,...,...,...,...,...
1028,hCoV-19/USA/CA-SEARCH-75725/2022,EPI_ISL_11103260,2022-01-18,BA.2,21L (Omicron),92109,3,San Diego
1029,hCoV-19/USA/CA-SEARCH-80026/2022,EPI_ISL_11534696,2022-01-05,BA.2,21L (Omicron),92122,1,San Diego
1030,hCoV-19/USA/CA-SEARCH-76579/2022,EPI_ISL_11146567,2022-01-05,BA.2,21L (Omicron),92071,1,Santee
1031,hCoV-19/USA/CA-SEARCH-114975/2022,EPI_ISL_9196061,2022-01-04,BA.2,21L (Omicron),92127,1,San Diego


In [17]:
weekly_data = zip_community_merge.groupby(["community", "nextclade_pango", "week"]).size().reset_index(name="cases").sort_values(["community", "week"])
weekly_data

,community,nextclade_pango,week,cases
0,Alpine,BA.2.12.1,19,1
1,Alpine,BA.2.12.1,20,1
2,Alpine,BA.2.12.1,22,1
3,Alpine,BA.2.12.1,23,2
6,Alpine,BA.5,23,1
...,...,...,...,...
316,Vista,BA.2.12.1,19,1
317,Vista,BA.2.12.1,20,1
315,Vista,BA.2,21,1
318,Vista,BA.2.12.1,21,1


In [18]:
weekly_first_appearance_merge = weekly_data.merge(grouped_data, how="left", on=["community", "nextclade_pango"])
weekly_first_appearance_merge

,community,nextclade_pango,week,cases,week_appeared
0,Alpine,BA.2.12.1,19,1,19
1,Alpine,BA.2.12.1,20,1,19
2,Alpine,BA.2.12.1,22,1,19
3,Alpine,BA.2.12.1,23,2,19
4,Alpine,BA.5,23,1,23
...,...,...,...,...,...
315,Vista,BA.2.12.1,19,1,19
316,Vista,BA.2.12.1,20,1,19
317,Vista,BA.2,21,1,9
318,Vista,BA.2.12.1,21,1,19


In [19]:
weekly_first_appearance_merge["growth_week"] = weekly_first_appearance_merge["week"] - weekly_first_appearance_merge["week_appeared"]
weekly_first_appearance_merge

,community,nextclade_pango,week,cases,week_appeared,growth_week
0,Alpine,BA.2.12.1,19,1,19,0
1,Alpine,BA.2.12.1,20,1,19,1
2,Alpine,BA.2.12.1,22,1,19,3
3,Alpine,BA.2.12.1,23,2,19,4
4,Alpine,BA.5,23,1,23,0
...,...,...,...,...,...,...
315,Vista,BA.2.12.1,19,1,19,0
316,Vista,BA.2.12.1,20,1,19,1
317,Vista,BA.2,21,1,9,12
318,Vista,BA.2.12.1,21,1,19,2


In [25]:
weekly_first_appearance_merge[weekly_first_appearance_merge["community"] == "San Diego"].sort_values(["nextclade_pango", "growth_week"])

,community,nextclade_pango,week,cases,week_appeared,growth_week
214,San Diego,BA.2,1,3,1,0
215,San Diego,BA.2,3,2,1,2
216,San Diego,BA.2,4,1,1,3
217,San Diego,BA.2,6,3,1,5
218,San Diego,BA.2,7,4,1,6
219,San Diego,BA.2,8,1,1,7
220,San Diego,BA.2,9,4,1,8
221,San Diego,BA.2,10,6,1,9
222,San Diego,BA.2,11,11,1,10
224,San Diego,BA.2,12,13,1,11


In [21]:
weekly_data.to_csv("../data/2022_sd_weekly_genome_counts.csv", index=False)